In [ ]:
!pip install evaluate
!pip install transformers
!pip install datasets
!pip install wandb
!pip install sacrebleu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 KB 12.0 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!nvidia-smi

Tue Mar  7 13:29:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    48W / 400W |      0MiB / 40960MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import datasets
import torch.nn as nn
import torch
import transformers
import logging
import numpy as np
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from transformers import AutoTokenizer, BartForConditionalGeneration
import evaluate
from datetime import datetime

In [ ]:
def create_logger():
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)
    formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')

    file_handler = logging.FileHandler(filename = "./log")
    file_handler.setFormatter(formatter)
    file_handler.setLevel(logging.INFO)
    logger.addHandler(file_handler)

    console = logging.StreamHandler()
    console.setLevel(logging.DEBUG)
    console.setFormatter(formatter)
    logger.addHandler(console)

    return logger

In [ ]:
logger = create_logger()

In [ ]:
train_dataset = datasets.load_from_disk('/content/drive/MyDrive/Hugging face/Translation/data/wmt19_train')['translation'][:10000]
val_dataset = datasets.load_from_disk('/content/drive/MyDrive/Hugging face/Translation/data/wmt19_val')['translation'][:1000]

In [ ]:
# class Dataset(Dataset):
#     def __init__(self,split):
#         self.dataset = datasets.load_from_disk("./data/wmt19")[split][:30000]
#     def __len__(self):
#         return len(self.dataset)
#     def __getitem__(self,i):
#         english = self.dataset['translation'][i]['en']
#         chinese = self.dataset['translation'][i]['zh']

#         return english, chinese

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50265, 768, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50265, 768, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 768)
      (layers): ModuleList(
        (0): BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (fc2): Linear(in_features=3072, out_features=768, bias=True)
          (final_layer_norm): LayerNorm((768,), eps=1e-05,

In [ ]:
def collate_fn(data):
    chinese = [i['zh'] for i in data]
    english = [i['en'] for i in data]

    source = tokenizer.batch_encode_plus(chinese,truncation = True, padding = 'max_length', max_length = 256, return_tensors = 'pt')
    target = tokenizer.batch_encode_plus(english,truncation = True, padding = 'max_length', max_length = 256, return_tensors = 'pt')

    source = source['input_ids']
    target = target['input_ids']

    source = source.to(dtype = torch.long)
    target = target.to(dtype = torch.long)

    source = source.to(device)
    target = target.to(device)

    return source,target



In [ ]:
def blue(logits,labels):
    bleu = evaluate.load("sacrebleu")
    tokens = logits.argmax(dim=-1)
    labels[labels == -100] = tokenizer.pad_token_id
    prediction = tokenizer.batch_decode(tokens,skip_special_tokens=True)
    targets = tokenizer.batch_decode(labels,skip_special_tokens=True)
    prediction = [pred.strip() for pred in prediction]
    targets = [label.strip() for label in targets]
    results = bleu.compute(predictions = prediction,references = targets)
    results = results['score']

    return results


In [ ]:
def train(epoch,dataloader,optimizer,scheduler,model,acc_grad):
    model.train()
    logger.info("Epoch:{} start".format(epoch+1))
    train_epoch_blue = []
    train_epoch_loss = []
    for i,(input_ids,targets) in tqdm(enumerate(dataloader),total = len(dataloader)):
        decoder_inputs = targets[:,:-1].contiguous()
        labels = targets[:,1:].contiguous()
        labels[labels == tokenizer.pad_token_id] = -100
        output = model(input_ids = input_ids,decoder_input_ids = decoder_inputs,labels = labels)
        loss = output.loss/acc_grad
        logits = output.logits
        loss.backward()
        batch_loss = loss.mean()
        train_epoch_loss.append(batch_loss)
        bleu_score = blue(logits,labels)
        train_epoch_blue.append(bleu_score)
        if (i+1)%acc_grad == 0:
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
    train_epoch_blue = sum(train_epoch_blue)/len(train_epoch_blue)
    train_epoch_loss = sum(train_epoch_loss)/len(train_epoch_loss)
    logger.info("Train Epoch {} finished; Train Loss:{}; Train Blue:{}".format(epoch+1,train_epoch_loss,train_epoch_blue))
    wandb.log({"Train loss": train_epoch_loss})
    wandb.log({"Train Blue": train_epoch_blue})
    wandb.log({"Encoder_lr":optimizer.state_dict()['param_groups'][0]['lr']})
    wandb.log({"Decoder_lr":optimizer.state_dict()['param_groups'][1]['lr']})
    wandb.log({"Lmhead_lr":optimizer.state_dict()['param_groups'][2]['lr']})
    wandb.log({"Other_lr":optimizer.state_dict()['param_groups'][3]['lr']})



In [ ]:
def val(epoch,dataloader,model,acc_grad,best_loss):
    val_epoch_blue = []
    val_epoch_loss = []
    model.eval()
    with torch.no_grad():
        for i,(input_ids,targets) in tqdm(enumerate(dataloader),total = len(dataloader)):
            decoder_inputs = targets[:,:-1].contiguous()
            labels = targets[:,1:].contiguous()
            labels[labels == tokenizer.pad_token_id] = -100
            output = model(input_ids = input_ids,decoder_input_ids = decoder_inputs,labels = labels)
            loss = output.loss/acc_grad
            logits = output.logits
            batch_loss = loss.mean()
            val_epoch_loss.append(batch_loss)
            bleu_score = blue(logits,labels)
            val_epoch_blue.append(bleu_score)

    val_epoch_blue = sum(val_epoch_blue)/len(val_epoch_blue)
    val_epoch_loss = sum(val_epoch_loss)/len(val_epoch_loss)
    if val_epoch_loss < best_loss:
        best_loss= val_epoch_loss
        model_name = "Best_loss.bin"
        torch.save(model,model_name)
        logger.info("Model is saved, with loss{}".format(best_loss))
    logger.info("Validation Epoch {} finished; Validation Loss:{}; Validation Blue:{}".format(epoch+1,val_epoch_loss,val_epoch_blue))
    wandb.log({"Validation loss": val_epoch_loss})
    wandb.log({"Validation Blue": val_epoch_blue})




In [ ]:
def test(epoch,model):
    word = ["我认为这件事没得商量，毕竟你有错在先。"]
    token = tokenizer.batch_encode_plus(word,padding = "max_length",max_length = 256,return_tensors = 'pt')
    token = token["input_ids"].to(device)
    output = model.generate(input_ids = token)
    output = tokenizer.decode(output.squeeze(0),skip_special_tokens=True)
    print(f"Test output:{output}")


In [ ]:
encoder_params = list(map(id,model.model.encoder.layers.parameters()))
decoder_params = list(map(id,model.model.decoder.layers.parameters()))
lm_params = [id(model.lm_head.weight)]
logit_params = [id(model.final_logits_bias)]
most_params = encoder_params+decoder_params+lm_params+logit_params
rest_params = filter(lambda p: id(p) not in most_params,model.parameters())

In [ ]:
from transformers import AdamW
import wandb

train_epoch =5
batch_size = 16
total_step = len(train_dataset)*train_epoch/batch_size
acc_grad = 4
optimizer = AdamW([{"params":model.model.encoder.layers.parameters(),'lr':1e-5},{"params":model.model.decoder.layers.parameters(),"lr":1.5e-4},
                   {"params":model.lm_head.weight,'lr':3e-4},{"params":model.final_logits_bias,"lr":3e-4},{"params":rest_params,"lr":1e-4}])
scheduler = transformers.get_linear_schedule_with_warmup(optimizer,num_warmup_steps = 0.1*total_step,num_training_steps = total_step)
train_loader = DataLoader(dataset = train_dataset, collate_fn = collate_fn, batch_size = batch_size,shuffle = True, drop_last = True)
val_loader = DataLoader(dataset = val_dataset, collate_fn = collate_fn, batch_size = batch_size, shuffle = True, drop_last = True)
wandb.init(project = 'Bart Translation',name='Test1',config = {"epochs":train_epoch,"batch_size":batch_size})


/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
from tqdm import tqdm
best_loss = 10
for epoch in range(train_epoch):
    epoch_start_time = datetime.now()
    train(epoch = epoch,dataloader=train_loader,optimizer=optimizer,scheduler=scheduler,model=model,acc_grad=acc_grad)
    val(epoch=epoch,dataloader=val_loader,model=model,acc_grad=acc_grad,best_loss=best_loss)
    test(epoch,model)
    epoch_end_time = datetime.now()
    logger.info("Epoch cost:{}".format(epoch_end_time - epoch_start_time))


2023-03-07 13:35:05,582 - INFO - Epoch:1 start
INFO:__main__:Epoch:1 start
100%|██████████| 625/625 [22:47<00:00,  2.19s/it]
2023-03-07 13:57:52,815 - INFO - Train Epoch 1 finished; Train Loss:1.154461145401001; Train Blue:0.3372209365023935
INFO:__main__:Train Epoch 1 finished; Train Loss:1.154461145401001; Train Blue:0.3372209365023935
100%|██████████| 62/62 [01:59<00:00,  1.93s/it]
2023-03-07 13:59:53,850 - INFO - Model is saved, with loss1.0791847705841064
INFO:__main__:Model is saved, with loss1.0791847705841064
2023-03-07 13:59:53,854 - INFO - Validation Epoch 1 finished; Validation Loss:1.0791847705841064; Validation Blue:0.39243487233355295
INFO:__main__:Validation Epoch 1 finished; Validation Loss:1.0791847705841064; Validation Blue:0.39243487233355295
/usr/local/lib/python3.8/dist-packages/transformers/generation/utils.py:1273: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`generation_config.max_length`). Controlling `ma

Test output:, for example, it is not surprising that the US has a large military presence in


100%|██████████| 625/625 [22:50<00:00,  2.19s/it]
2023-03-07 14:22:44,427 - INFO - Train Epoch 2 finished; Train Loss:0.9393688440322876; Train Blue:0.43351660113174595
INFO:__main__:Train Epoch 2 finished; Train Loss:0.9393688440322876; Train Blue:0.43351660113174595
100%|██████████| 62/62 [01:59<00:00,  1.93s/it]
2023-03-07 14:24:45,793 - INFO - Model is saved, with loss1.0964041948318481
INFO:__main__:Model is saved, with loss1.0964041948318481
2023-03-07 14:24:45,796 - INFO - Validation Epoch 2 finished; Validation Loss:1.0964041948318481; Validation Blue:0.33931168655797245
INFO:__main__:Validation Epoch 2 finished; Validation Loss:1.0964041948318481; Validation Blue:0.33931168655797245
2023-03-07 14:24:46,012 - INFO - Epoch cost:0:24:51.875265
INFO:__main__:Epoch cost:0:24:51.875265
2023-03-07 14:24:46,015 - INFO - Epoch:3 start
INFO:__main__:Epoch:3 start


Test output:, for example, is that the US is not the only country in the world that


  2%|▏         | 14/625 [00:32<23:49,  2.34s/it]


KeyboardInterrupt: ignored

In [ ]:
for x,y in train_loader:
    a = x
    b = y
    break

In [ ]:
#tokenizer.batch_decode(b,skip_special_tokens = True

{'bleu': 100.00000000000004}

7it [00:00, 2214.86it/s]

(0, 3)
(1, 4)
(2, 5)
(3, 6)
(4, 3)
(5, 4)
(6, 4)
